In [ ]:
from sentence_transformers import SentenceTransformer, util
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os
import torch
#.venv/NPL Projects/Text to image search/photos

In [ ]:
img_names = list(glob.glob('.venv/NPL Projects/Text to image search/photos/*.jpg'))[:2000]

print(len(img_names))

In [ ]:
img_model = SentenceTransformer('clip-VIT-B-32')

In [ ]:
img_embeddings = img_model.encode([Image.open(img) for img in img_names],convert_to_tensor=True,show_progress_bar=True,batch_size=32)

In [ ]:
embeddings = img_embeddings
threshold = 0.8
min_com_size = 10
max_com_size = 1000

cos_scores = util.cos_sim(embeddings, embeddings)

In [ ]:
cos_scores.shape

In [ ]:
top_k_vals,_ = cos_scores.topk(min_com_size)

In [ ]:
top_k_vals.shape

In [ ]:
extracted_communities = []

for i in range(len(top_k_vals)):
    if top_k_vals[i][-1] > threshold:
        new_cluster = top_k_vals[i]
        
        top_val_large, top_idx_large = cos_scores[i].topk(k=max_com_size,largest=True)
        top_idx_large = top_idx_large.tolist()
        top_val_large = top_val_large.tolist()
        new_cluster = []
        if top_val_large[-1] < threshold:
            for idx,val in zip(top_idx_large,top_val_large):
                if val < threshold:
                    break
                new_cluster.append(idx)
        else:
            for idx,val in enumerate(cos_scores[i].tolist()):
                if val >= threshold:
                    new_cluster.append(idx)
                    
        extracted_communities.append(new_cluster)
    
extracted_communities = sorted(extracted_communities, key=lambda x: len(x), reverse=True)

In [ ]:
extracted_communities

In [ ]:
len(extracted_communities)

In [ ]:
unique_com = []
extracted_ids = set()

for community in extracted_communities:
    add_cluster = True
    for idx in community:
        if idx in extracted_ids:
            add_cluster = False
            break
    if add_cluster:
        unique_com.append(community)
        for idx in community:
            extracted_ids.add(idx)

In [ ]:
len(unique_com)

In [ ]:
for cluster in unique_com[0:10]:
    print('\n Cluster size: ', len(cluster))
    for idx in cluster[0:10]:
        img_path = img_names[idx]
        im = Image.open(img_path).convert('RGB')
        plt.axis('off')
        plt.imshow(im)
        plt.show()
        plt.figure()